# Installation

In [3]:
!pip install -q yt-dlp
!pip install -q git+https://github.com/openai/whisper.git
!pip install -U deeplake

DEPRECATION: celery 5.2.0 has a non-standard dependency specifier pytz>dev. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: celery 5.2.0 has a non-standard dependency specifier pytz>dev. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


     ------------------------------------ 580.3/580.3 kB 493.2 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 99.8/99.8 kB 1.4 MB/s eta 0:00:00
  Created wheel for deeplake: filename=deeplake-3.8.9-py3-none-any.whl size=699749 sha256=c71c80b29fae78576eda95bf8cecc8265027b468a7bc50b0283fdda30101f4c5
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\93\f5\10\9ba3d459167ce0cbe4f8b4d4dcc96e1de28351ae1e2041d643
Successfully built deeplake
  Attempting uninstall: deeplake
    Found existing installation: deeplake 3.7.1
    Uninstalling deeplake-3.7.1:
      Successfully uninstalled deeplake-3.7.1


DEPRECATION: celery 5.2.0 has a non-standard dependency specifier pytz>dev. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


# Imports

In [7]:
import yt_dlp
import whisper
import textwrap
from langchain import OpenAI, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA

# Video Downloader

In [8]:
# Video info Function
def video_downloader(urls, job_id):
    # for storing information of videos
    video_info = []

    for i, url in enumerate(urls):
        file_name = f'./{job_id}_{i}.mp4'
        ydl_opts = {
            'outtmpl' : file_name,
            'quite' : True,
            'format' : "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp3]"
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # The line below is downloading the video
            result = ydl.extract_info(url, download=True)
            author = result.get("uploader","")
            title = result.get("title","")

        video_info.append((file_name, title, author))
    return video_info

In [10]:
# Download
url = ["https://www.youtube.com/watch?v=0FX4jVAL1eg&pp=ygUJam9lIHJvZ2Fu"]
video_details = video_downloader(url, "A1")

[youtube] Extracting URL: https://www.youtube.com/watch?v=0FX4jVAL1eg&pp=ygUJam9lIHJvZ2Fu
[youtube] 0FX4jVAL1eg: Downloading webpage
[youtube] 0FX4jVAL1eg: Downloading ios player API JSON
[youtube] 0FX4jVAL1eg: Downloading android player API JSON
[youtube] 0FX4jVAL1eg: Downloading m3u8 information
[info] 0FX4jVAL1eg: Downloading 1 format(s): 616+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 71
[download] Destination: A1_0.f616.mp4
[download] 100% of  100.33MiB in 00:00:58 at 1.72MiB/s                    
[download] Destination: A1_0.f140.m4a
[download] 100% of    6.41MiB in 00:00:01 at 4.98MiB/s   
[Merger] Merging formats into "A1_0.mp4"
Deleting original file A1_0.f616.mp4 (pass -k to keep)
Deleting original file A1_0.f140.m4a (pass -k to keep)


# Transcription

In [12]:
# Loading Whisper model
model = whisper.load_model("base")
result = model.transcribe("A1_0.mp4")
print(result["text"])

 The gel-rogan experience. The stuff in the sea very alienly on the back. Yeah. I agree. You know, they did some study recently where they showed that jellyfish, despite not having a brain, they seemed to be able to learn. Now they're not trying to figure out, well, how have they learned? I know how they learn. Vibes. Vibes, man. They feel the vibes. They feel the vibes. Yeah, because they don't have a brain to feel the vibes. What if we live in a vibrational universe where everything is frequency, it would make sense. They feel the vibes. They're not down there. They just go with what the flow is. They're very jelly-like. They just go with whatever the energy is doing already. That's why it seems like they know. You guys seem like you came up with this theory together and you discussed this? Well, I just thought of it right now and Sarah came on board with me and I feel like we're on to something. Yeah, we're on to something. Well, no, I think we live in a vibrational universe and thi

In [13]:
# Writing Transcription
with open("./DocumentOne.txt", 'w') as f:
    f.write(result['text'])

In [14]:
# Initializing the LLM
llm = OpenAI(model_name='text-davinci-003', temperature=0)

In [15]:
# Initializing the Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

In [16]:
# Splitting the text
with open('./DocumentOne.txt', 'r') as file:
    text = file.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:4]]

In [17]:
# Initializing the Chain with different Chain types
chain = load_summarize_chain(llm=llm, chain_type='map_reduce') # Use chain types "refine"

summary = chain.run(docs)

# Text Wrap
wrap = textwrap.fill(summary, width=100)
print(wrap)

 This text discusses the intelligence of jellyfish and octopi, two creatures that have adapted to
their environment by becoming poisonous or aware of danger. It mentions the box jellyfish and the
blue-ringed octopus, which are both found in Australia, and the puffer fish, which contains a toxin
that can kill 26 full adults with one bite. It also discusses how octopi are intelligent creatures,
capable of memorizing and opening mason jars and unlocking doors.


In [18]:
prompt_template = """
    Write a concise summary of the following:

    
    "{text}"


    CONCISE SUMMARY IN BULLET POINTS
"""

Bullet_point_template = PromptTemplate(
    template=prompt_template,
    input_variables=["text"]
)

returnn = load_summarize_chain(llm=llm, chain_type="stuff", prompt=Bullet_point_template)

summary1 = returnn.run(docs)
wrap1 = textwrap.fill(summary1, width=1000, break_long_words=False, replace_whitespace=False)
print(wrap1)

    - Jellyfish have been found to be able to learn despite not having a brain
    - It is suggested that this is because they are able to feel the vibes of the universe
    - Box jellyfish are particularly dangerous and have a toxin that is 1,200 times more toxic than cyanide
    - Octopi are also very intelligent and have been found to be able to open mason jars, unlock doors, and memorize things


# Adding Transcripts to DeepLake

In [19]:
# Login
!activeloop login -u vishnusharma7 -p <password>

Successfully logged in to Activeloop.


In [20]:
# Embedding & Upload
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
activeloop_id = 'vishnusharma7'
dataset = 'VideoSummarizer2'
dataset_path = f"hub://{activeloop_id}/{dataset}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/vishnusharma7/VideoSummarizer2


hub://vishnusharma7/VideoSummarizer2 loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [02:42<00:00


Dataset(path='hub://vishnusharma7/VideoSummarizer2', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (4, 1536)  float32   None   
    ids      text     (4, 1)      str     None   
 metadata    json     (4, 1)      str     None   
   text      text     (4, 1)      str     None   


['f08677c3-918f-11ee-b60b-001e64f80f6a',
 'f08677c4-918f-11ee-98a7-001e64f80f6a',
 'f08677c5-918f-11ee-b082-001e64f80f6a',
 'f08677c6-918f-11ee-b8f9-001e64f80f6a']

# Retriever

In [21]:
# Retriever Defining
retriver = db.as_retriever()
retriver.search_kwargs['distance_metric'] = 'cos'
retriver.search_kwargs['k'] = 4

In [26]:
# Retriver Template
template = """ Use the following pieces of transcripts from a database to answer the question in bullet point and summarized.
And if the answer is not in the given transcript, then just say you don't know, Don't try to forge an answer outside of the
provided retriever.

{context}

Question = {question}
Answer in summarized bullet points:
"""

prompt_template = PromptTemplate(template=template, input_variables=['context', 'question'])

In [27]:
# Retriever Initialization
chain_type_kwargs = {"prompt" : prompt_template}
qa = RetrievalQA.from_chain_type(llm = llm,
                               chain_type="stuff",
                               retriever = retriver,
                               chain_type_kwargs=chain_type_kwargs)

In [28]:
# Inference
print(qa.run("What are the signs of top 1% humans?")) # Why Jellyfish is considered intelligent?

- Thinking and creating the world
- Being aware of danger
- Memorizing things
- Being able to open mason jars
- Being able to unlock doors
- Being able to figure out how things work
